In [ ]:
import os
import re
import pandas as pd
from pyproj import Transformer
import mgrs

In [ ]:
# Función para convertir coordenadas de formato "10kmE<easting>N<northing>" a MGRS
def convert_10km_to_mgrs(coord_str: str) -> str:
    # Extraer índices de easting y northing
    match = re.match(r"10kmE(\d+)N(\d+)", coord_str)
    if not match:
        raise ValueError("El formato debe ser '10kmE<easting>N<northing>'")
    easting_idx = int(match.group(1))
    northing_idx = int(match.group(2))

    # Convertir índices a metros
    easting = easting_idx * 10000
    northing = northing_idx * 10000

    # Transformar de ETRS89/LAEA (EPSG:3035) a geográficas (EPSG:4326)
    transformer = Transformer.from_crs("EPSG:3035", "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(easting, northing)

    # Convertir a MGRS con precisión de 1 km
    mgrs_converter = mgrs.MGRS()
    mgrs_code = mgrs_converter.toMGRS(lat, lon, MGRSPrecision=1)
    return mgrs_code


# Función para ajustar valores del tipo "30TVL33" o "30TVL3231"
def ajustar_valor(cadena: str) -> str:
    # Procesa solo si la cadena empieza por alguno de los prefijos deseados
    if not (cadena.startswith("30") or cadena.startswith("29") or cadena.startswith("28") or cadena.startswith("31")):
        return cadena
    
    # Patrón 1: formato "<prefijo><letras><2 dígitos>" -> se deja igual.
    patron1 = r"^((?:30|29|28|31)[A-Z]+)(\d{2})$"
    # Patrón 2: formato "<prefijo><letras><2 dígitos><2 dígitos>" -> se elimina los dos últimos dígitos.
    patron2 = r"^((?:30|29|28|31)[A-Z]+)(\d{2})(\d{2})$"
    
    if re.match(patron1, cadena):
        return cadena
    elif re.match(patron2, cadena):
        m = re.match(patron2, cadena)
        return m.group(1) + m.group(2)
    else:
        return cadena


i = 0

# Procesar cada archivo Excel en la carpeta
for archivo in os.listdir('./EIDOS_ubicacion'):
    if archivo.ends_with('.xlsx'):
        i += 1
        df = pd.read_excel(f"EIDOS_ubicacion/{archivo}")
        
        for idx, row in df.iterrows():
            valor = row["cuadr"]
            # Si el valor es de tipo "10km...", aplicar la conversión a MGRS
            if valor.startswith("10km"):
                try:
                    nuevo_valor = convert_10km_to_mgrs(valor)
                    df.loc[idx, "cuadr"] = nuevo_valor
                except Exception as e:
                    print(f"Error en archivo {archivo}, fila {idx} con valor '{valor}': {e}")
            df.loc[idx, "cuadr"] = ajustar_valor(valor)

        df = df.drop_duplicates(subset=['idtaxon', 'cuadr'], keep="first")
        df = df.sort_values(by="idtaxon")
        df.to_excel(f"EIDOS_ubicacion/mismo_cud_sin_repetidos_{i}.xlsx", index=False)
        os.remove(f"EIDOS_ubicacion/{archivo}")

In [ ]:
dfs = []
total_length = 0
LENGTH_LIMIT = 100000
i = 1

# Combinar los archivos excel, ahora sin repetidos, en archivos mas grandes
for file in os.listdir('./EIDOS_ubicacion'):
    if file.endswith('.xlsx') and file.startswith("mismo_cud_sin_repetidos_"):
        print(f"Leyendo {file}...", end='\r')
        dfs.append(pd.read_excel(f"EIDOS_ubicacion/{file}"))
        total_length += len(dfs[-1])
        if total_length >= LENGTH_LIMIT:
            df = pd.concat(dfs)
            df.sort_values(by="idtaxon")
            print(f"Exportando archivo combinado {i}...{' '*10}", end='\r')
            df.to_excel(f"EIDOS_ubicacion/EIDOS_ubicacion_final_{i}.xlsx", index=False)
            i += 1
            dfs = []
            total_length = 0
if total_length > 0:
    df = pd.concat(dfs)
    df.sort_values(by="idtaxon")
    print(f"Exportando archivo combinado {i}...{' '*10}", end='\r')
    df.to_excel(f"EIDOS_ubicacion/EIDOS_ubicacion_final_{i}.xlsx", index=False)

# Eliminar archivos anteriores
print(f"Eliminando archivos residuales...{' '*10}", end='\r')
for file in os.listdir('./EIDOS_ubicacion'):
    if file.endswith('.xlsx') and not file.startswith("EIDOS_ubicacion_final_"):
        os.remove(f"EIDOS_ubicacion/{file}")